the competition and data is available in Kaggle at: https://www.kaggle.com/competitions/playground-series-s5e1/overview

# Kaggle Sticker 3 Years Sales Forecast Competition

end-to-end using Prophet for regression problem 

# Import Libraries

In [ ]:
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error
import logging
import warnings

logging.getLogger('prophet').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows',None)

In [ ]:
zip_path = os.path.join(os.getcwd(),'dataset//playground-series-s5e1.zip')
unzip_export_path = os.path.join(os.getcwd(),'dataset')

# Open the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # List the contents of the ZIP file
    print("Contents of the ZIP file:")
    print(zip_ref.namelist())

    # Extract all files
    zip_ref.extractall(unzip_export_path)
    print("\nFiles extracted.")

# Load Datasets and Apply Initial Clean Up

In [ ]:
train_path = os.path.join(os.getcwd(),'dataset//train.csv')
test_path = os.path.join(os.getcwd(),'dataset//test.csv')

train0 = pd.read_csv(train_path)
test0 = pd.read_csv(test_path)

display(
    train0.tail(),
    train0.shape[0],
    train0.isna().sum(),
    train0.dtypes,
    test0.head(),
    test0.shape[0],
    test0.isna().sum(),
    test0.dtypes
)

Now let's fill the NaN fields with 0, and convert date columns into datetime

In [ ]:
train = train0.copy()
test = test0.copy()

train = train.fillna(0)
test = test.fillna(0)

train['date'] = pd.to_datetime(train['date'],yearfirst=True)
test['date'] = pd.to_datetime(test['date'],yearfirst=True)

print(train.dtypes,train.isna().sum())
print(test.dtypes,test.isna().sum())

display(train.head(),test.head())

In [ ]:
# Let's understand the date range 
print(train['date'].min(), train['date'].max())
print(test['date'].min(),test['date'].max())

# Data Preprocessing

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import zscore
import pandas as pd  

class DataPreprocessing:
    def __init__(self, df):
        self.df = df
    
    def groupby_product(self):
        # Creating a 'primary_key' to identify the combination of country, store, and product
        self.df['primary_key'] = self.df['country'].astype(str) + '_' + self.df['store'].astype(str) + '_' + self.df['product'].astype(str)
        # Grouping by primary_key and date, summing the number of units sold
        df_grouped = self.df.groupby(by=['primary_key', 'date'], as_index=False)['num_sold'].sum().reset_index()
        df_grouped['num_sold'] = df_grouped['num_sold'].astype(int)  
        return df_grouped


class EDA_Plot_And_Outlier_Check:
    def __init__(self, grouped_df, Z_score):
        self.grouped_df = grouped_df
        self.Z_score = Z_score

    def plot(self):
        outlier_dict = {}  # To store outliers for each product
        
        for product in self.grouped_df['primary_key'].unique():
            df1 = self.grouped_df[self.grouped_df['primary_key'] == product]
            
            # Plot the historical sales data
            plt.plot(df1['date'], df1['num_sold'], label='Historical Sales')
            plt.title(f'{product} - Historical Sales by Date')
            plt.xlabel('Date')
            plt.ylabel('Number of Units Sold')
            plt.legend()
            plt.show()
            
            # Compute z-score and identify outliers
            z_scores = zscore(df1['num_sold'])
            outliers = df1[abs(z_scores) > self.Z_score]
            outlier_dict[product] = outliers[['date', 'num_sold']] 

            # # Print outliers for each product
            # print(f'Outliers for {product}:')
            # print(outliers[['date', 'num_sold']])
        
        # Convert outlier_dict to a DataFrame after collecting all outliers
        outlier_df = pd.concat(outlier_dict.values(), keys=outlier_dict.keys()).reset_index()
        outlier_df.rename(columns={'level_0': 'primary_key'}, inplace=True)

        return outlier_df 


In [ ]:
Preprocessor = DataPreprocessing(train)
df_train = Preprocessor.groupby_product()
j=0
for i in df_train['primary_key'].unique():
    if df_train[df_train['primary_key']==i]['num_sold'].sum() == 0:
       df_train = df_train[~(df_train['primary_key']==i)]
       j+=1
print(f'dropped {j} primary keys as there are no sales!')
df_train = df_train.reset_index(drop=True)
df_train.head()
    

In [ ]:
EDA_Plotter = EDA_Plot_And_Outlier_Check(df_train,3)
outlier_df = EDA_Plotter.plot()

In [ ]:
outlier_df.head()

# Exploratory Data Analysis Result:

Based on initial analysis, for all products, it is clear that Christmas period explains most of their outliers. I will now handle few outliers outside of the period for individual product as they seems not to be repetitive events.

Outliers to handle: 

Kerneler Dark Mode - date == 2011-04-24,2011-05-01
Kerneler - date == 2011-09-11
Kaggle Tiers - date == 2011-11-06

Instead of just removal, I will populate them with their previous 7 days sales' average


# Remove Outliers

In [ ]:
df_train.head(2)

In [ ]:
outlier_df.shape[0]

In [ ]:

df_train.isna().sum()

In [ ]:
class Outlier_Replace:
    def __init__(self, df, prev_days_avg, outlier_df):
        self.df = df
        self.outlier_df = outlier_df
        self.prev_days_avg = prev_days_avg

    def replace_avg(self):
        for key in self.outlier_df['primary_key'].unique(): 
            df3 = self.df[self.df['primary_key'] == key]
            outlier_df1 = self.outlier_df[self.outlier_df['primary_key'] == key]
            
            for date in outlier_df1['date']:
                # Get the original outlier value
                original_outlier_value = df3[df3['date'] == date]["num_sold"].values[0]
                
                # Get previous days up to the given date
                previous_days = df3[df3['date'] < date]['date']
                df4 = df3[df3['date'].isin(previous_days)]
                prev_avg = df4.tail(self.prev_days_avg)['num_sold'].mean()  
                
                # Check if the previous days' average is NaN, and if so, calculate the next 7 days' average
                if pd.isna(prev_avg):
                    next_days = df3[df3['date'] > date]['date']
                    df5 = df3[df3['date'].isin(next_days)]
                    next_avg = df5.head(self.prev_days_avg)['num_sold'].mean()  
                    if pd.isna(next_avg):
                        next_avg = 0  # Fallback in case both are NaN
                    self.df.loc[(self.df['date'] == date) & (self.df['primary_key'] == key), 'num_sold'] = next_avg
                    print(f'For {key}, outlier value {original_outlier_value.astype(int)} at {date.date()} has been replaced '
                          f'with the next {self.prev_days_avg} days average: {next_avg.astype(int)}')
                else:
                    # If previous average is valid, use it
                    self.df.loc[(self.df['date'] == date) & (self.df['primary_key'] == key), 'num_sold'] = prev_avg
                    print(f'For {key}, outlier value {original_outlier_value.astype(int)} at {date.date()} has been replaced '
                          f'with the previous {self.prev_days_avg} days average: {prev_avg.astype(int)}')
                
        return self.df  


Outlier_Replace1 = Outlier_Replace(df_train, 7, outlier_df)
df_train_OR = Outlier_Replace1.replace_avg()


Now let's have a look at the plots after the iregular outliers have been handled

In [ ]:
EDA_Plotter_OR = EDA_Plot_And_Outlier_Check(df_train_OR,3)
EDA_Plotter_OR.plot()

I am happy about the pattern now to proceed with model training 

# Forecast

In [ ]:
df_train_OR.head(1)


In [ ]:
train_data = df_train_OR.copy()
train_data = train_data[['primary_key','date','num_sold']]
train_data.rename(columns = {'date':'ds','num_sold':'y'},inplace=True)
train_data.head()

In [ ]:
FORECAST_RESULT = []
for product in train_data['primary_key'].unique():
    df=train_data[train_data['primary_key']==product].iloc[:,1:]
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=1095)
    forecast = m.predict(future)
    forecast1= forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
    forecast1['primary_key'] = product
    fig2 = m.plot_components(forecast)
    fig2.suptitle(f'{product} - Forecast Components Analysis', fontsize=16, y=1.02)
    FORECAST_RESULT.append(forecast1)

Forecast_Result = pd.concat(FORECAST_RESULT,ignore_index=True)


In [ ]:
class PlotForecastResult:
    def __init__(self, forecast, train_data, fig_size=(10, 6)):
        """
        Initializes the class with forecast data, training data, and figure size.

        :param forecast: DataFrame containing forecasted results.
        :param train_data: DataFrame containing training data.
        :param fig_size: Tuple specifying the figure size (width, height).
        """
        self.forecast = forecast
        self.train_data = train_data
        self.fig_size = fig_size

    def plotter(self):
        """
        Plots historical and forecasted data for each product.
        """
        for product in self.forecast['primary_key'].unique():
            df1 = self.forecast[self.forecast['primary_key'] == product]
            df2 = self.train_data[self.train_data['primary_key'] == product]
            
            plt.figure(figsize=self.fig_size) 
            
            plt.plot(df1['ds'], df1['yhat'], color='green', label='Forecast')
            plt.plot(df2['ds'], df2['y'], color='grey', label='Historical')
            plt.title(f'{product} - Historical and Forecasted Sales by Date')
            plt.xlabel('Date')
            plt.ylabel('Unit Sales')
            plt.legend()
            plt.show()

            plt.clf()


In [ ]:
class MAPECalculator:
    def __init__(self, forecast_result, train_data):
        """
        Initializes the MAPECalculator with forecast results and training data.
        
        :param forecast_result: DataFrame containing forecasted results.
        :param train_data: DataFrame containing training data.
        """
        self.forecast_result = forecast_result
        self.train_data = train_data
        self.result = None
        self.mape_train = None

    def prepare_data(self):
        """
        Prepares the data by merging forecast results with training data 
        and calculates MAPE for each row.
        """
        self.result = self.forecast_result[['primary_key', 'ds', 'yhat']].merge(
            self.train_data, on=['primary_key', 'ds'], how='left'
        )
        self.result.rename(columns={'ds': 'date', 'yhat': 'forecast', 'y': 'actual'}, inplace=True)
        self.result = self.result[['primary_key', 'date', 'actual', 'forecast']]
        self.result = self.result[~self.result['actual'].isna()]
        self.result['forecast'] = self.result['forecast'].astype(int)
        self.result['actual'] = self.result['actual'].astype(int)
        self.result['MAPE'] = (abs(self.result['forecast'] - self.result['actual']) / self.result['actual']).round(2)

    def calculate_mape(self):
        """
        Calculates the mean MAPE for each product and stores the results.
        """
        mape_train = []
        for product in self.result['primary_key'].unique():
            result1 = self.result[self.result['primary_key'] == product]
            mape = result1['MAPE'].mean().round(2)
            mape_train.append({'primary_key': product, 'mape': mape})
        self.mape_train = pd.DataFrame(mape_train)

    def get_mape(self):
        """
        Returns the DataFrame containing MAPE values for each product.
        """
        return self.mape_train

    def process(self):
        """
        Executes the entire MAPE calculation workflow.
        """
        self.prepare_data()
        self.calculate_mape()
        return self.get_mape()

In [ ]:
forecast_result = Forecast_Result 
train_data = train_data 

mape_calculator = MAPECalculator(forecast_result, train_data)
mape = mape_calculator.process()

print(mape)

In [ ]:

class PlotForecastResult:
    def __init__(self, forecast, train_data, fig_size=(10, 6)):
        """
        Initializes the class with forecast data, training data, and figure size.

        :param forecast: DataFrame containing forecasted results.
        :param train_data: DataFrame containing training data.
        :param fig_size: Tuple specifying the figure size (width, height).
        """
        self.forecast = forecast
        self.train_data = train_data
        self.fig_size = fig_size

    def plotter(self):
        """
        Plots historical and forecasted data for each product.
        Also displays the MAPE (forecast accuracy) for each product.
        """
        for product in self.forecast['primary_key'].unique():
            df1 = self.forecast[self.forecast['primary_key'] == product]
            df2 = self.train_data[self.train_data['primary_key'] == product]
            
            # Calculate MAPE for the product
            result = df1[['ds', 'yhat']].merge(df2[['ds', 'y']], on='ds', how='left')
            result['MAPE'] = (abs(result['yhat'] - result['y']) / result['y']).mean().round(2)

            mape_value = result['MAPE'][0]  
            
            # Create figure with specified size
            plt.figure(figsize=self.fig_size)
            
            # Plot the forecasted and historical data
            plt.plot(df1['ds'], df1['yhat'], color='green', label='Forecast')
            plt.plot(df2['ds'], df2['y'], color='grey', label='Historical')
            
            # Add the MAPE in the plot's title
            plt.title(f'{product} - Historical and Forecasted Sales by Date\nMAPE: {mape_value * 100}%')
            plt.xlabel('Date')
            plt.ylabel('Unit Sales')
            plt.legend()
            plt.show()

            plt.clf()



In [ ]:
plotter = PlotForecastResult(Forecast_Result, train_data, fig_size=(12, 8))
plotter.plotter()

# Create submission.csv to to complete the Kaggle compitition

In [ ]:
forecast_result.tail()
final = forecast_result[['primary_key','ds','yhat']]
final.rename(columns={'ds':'date','yhat':'forecasted_num_sold'},inplace=True)
final['forecasted_num_sold'] = final['forecasted_num_sold'].astype(int)
final.tail()


In [ ]:
test = test0
test['primary_key'] = test['country'].astype(str)+'_'+test['store'].astype(str)+'_'+test['product'].astype(str)
test['date'] = pd.to_datetime(test['date'],yearfirst=True)
test.head()

In [ ]:
submission_csv = final.merge(test,on=['primary_key','date'],how='right')
submission_csv['forecasted_num_sold'] = submission_csv['forecasted_num_sold'].fillna(0)
submission_csv = submission_csv[['id','forecasted_num_sold']]
submission_csv.rename(columns={'forecasted_num_sold':'num_sold'},inplace=True)
display(submission_csv.tail())
path = os.path.join(os.getcwd(),'submission.csv')
submission_csv.to_csv(path,index=False)

In [ ]:
submission_csv

Now, i will submit it into kaggle competition!